In [1]:
!test -f test6.grib || wget https://github.com/ecmwf/emohawk/raw/main/docs/examples/test6.grib

## Loading GRIB data from a stream

In [2]:
import emohawk

emohawk can load GRIB data from a **stream**, which can be an FDB stream, a standard Python IO stream or any object implementing the necessary stream methods. 

For simplicity, in this notebook we will use a file stream to demonstrate the usage of streams.

### Single iteration

We create a stream from a file containing 6 GRIB fields by simply calling *open()*. It returns an io.BufferedReader object (a file stream).

In [3]:
stream = open("test6.grib", "rb")

We load it into emohawk by using the **single_iter=True** (default) option. With this *fs* will merely act like a stream iterator allowing only one full iteration over the data. 

In [4]:
fs = emohawk.load_from("stream", stream, single_iter=True)

At this point nothing is read from the stream. As we progressing with the iteration GribField objects are created then get deleted when going out of scope. As a result there is only one GRIB message is kept in memory at a time.

In [5]:
for f in fs:
    # f is GribField object. It gets deleted when going out of scope
    print(f)
    print(f"  mean={f.to_numpy().mean()}")

GribField(t,1000,20180801,1200,0,0)
  mean=279.70703560965404
GribField(u,1000,20180801,1200,0,0)
  mean=-0.38211858840215773
GribField(v,1000,20180801,1200,0,0)
  mean=-0.07128106980096727
GribField(t,850,20180801,1200,0,0)
  mean=272.72964550199964
GribField(u,850,20180801,1200,0,0)
  mean=0.3397187732514881
GribField(v,850,20180801,1200,0,0)
  mean=-0.19617789132254465


Having finished the iteration there is no data available any longer in *fs*.  We can close the stream:

In [6]:
stream.close()

### Storing each GRIB message in memory

The other usage of streams is setting **single_iter=False** in *load_from()*:

In [7]:
stream = open("test6.grib", "rb")
fs = emohawk.load_from("stream", stream, single_iter=False)

The resulting emohawk object is empty at this point. However, as soon as we call any method on it or start the iteration all the GRIB messages are loaded from the stream and will be stored in memory as long as *fs* exists.

We can call all the standard emohawk methods on *fs*:

In [8]:
len(fs)

6

In [9]:
fs.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
2,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
3,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
4,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
5,ecmf,v,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [10]:
a = fs.sel(param="t")
a.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [11]:
a = a.to_xarray()
a

<xarray.Dataset>
Dimensions:        (number: 1, time: 1, step: 1, isobaricInhPa: 2, latitude: 7,
                    longitude: 12)
Coordinates:
  * number         (number) int64 0
  * time           (time) datetime64[ns] 2018-08-01T12:00:00
  * step           (step) timedelta64[ns] 00:00:00
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 850.0
  * latitude       (latitude) float64 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude      (longitude) float64 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
    valid_time     (time, step) datetime64[ns] ...
Data variables:
    t              (number, time, step, isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-02-23T15:35 GRIB to CDM+CF via cfgrib-0.9.1...

We close the stream:

In [12]:
stream.close()